# <center> VCR Modeling
<center> Anqi Zhu

In [18]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.utils.fixes import loguniform
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV

from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC, SVC
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

from sklearn.decomposition import PCA, KernelPCA
from sklearn import metrics


In [2]:
# set random seed for reproducibility
import random
RANDOM_SEED = 42
random.seed(RANDOM_SEED)

## Data reading and initial checking 

In [3]:
df = pd.read_csv("VCR_clean_real.csv")
df.head()

,destination,passanger,weather,temperature,time,coupon,expiration,gender,age,maritalStatus,...,occupation,income,Bar,CoffeeHouse,CarryAway,RestaurantLessThan20,Restaurant20To50,direction_same,to_Coupon,Y
0,No Urgent Place,Alone,Sunny,55,2PM,Restaurant(<20),1d,Male,21,Single,...,Architecture & Engineering,$62500 - $74999,never,less1,4~8,4~8,less1,0,5-15,1
1,No Urgent Place,Friend(s),Sunny,80,10AM,Coffee House,2h,Male,21,Single,...,Architecture & Engineering,$62500 - $74999,never,less1,4~8,4~8,less1,0,5-15,0
2,No Urgent Place,Friend(s),Sunny,80,10AM,Bar,1d,Male,21,Single,...,Architecture & Engineering,$62500 - $74999,never,less1,4~8,4~8,less1,0,5-15,1
3,No Urgent Place,Friend(s),Sunny,80,10AM,Carry out & Take away,2h,Male,21,Single,...,Architecture & Engineering,$62500 - $74999,never,less1,4~8,4~8,less1,0,15-25,0
4,No Urgent Place,Friend(s),Sunny,80,2PM,Coffee House,1d,Male,21,Single,...,Architecture & Engineering,$62500 - $74999,never,less1,4~8,4~8,less1,0,5-15,0


In [ ]:
df.info()

## EDA

### Univariate analyis 

In [ ]:
fig, axes = plt.subplots(11, 2, figsize=(25,50))
axes = axes.flatten()

for ax, col in zip(axes, df.columns):
    sns.countplot(y=col, data=df, ax=ax, order=df[col].value_counts().index)

plt.tight_layout()
plt.show()

### Bivariate analysis

In [ ]:
fig, axes = plt.subplots(11, 2, figsize=(25, 50))
axes = axes.flatten()

for ax, col in zip(axes, df.drop('Y', axis=1).columns):
    sns.countplot(y=col, hue='Y', data=df, ax=ax)
plt.tight_layout()
plt.show()

**Note**

Remember to include important features based on EDA to models.

## Preprocessing

### Split features and target

In [4]:
X = df.drop(columns="Y", axis=1)
y = df["Y"]

In [ ]:
for i in X.columns:
    print(i, X[i].unique())

In [5]:
ord_cols = ['temperature', 'time', 'expiration', 'age', 'education', 'income', 'to_Coupon']
nom_cols = ['destination', 'passanger', 'weather', 'coupon', 'gender', 'maritalStatus', 'has_children', 
            'occupation', 'direction_same']
frequency_cols = ['Bar', 'CoffeeHouse', 'CarryAway','RestaurantLessThan20', 'Restaurant20To50']

def ordinal_encoding(df):
    '''
    Encode ordinal categorical features into ordinal numerical values(0,1,2,3,,,) - instead one-hot encoding.
    '''
    df_copy = df.copy()
    
    temperature_map = {30: 0, 55: 1, 80: 2}
    time_map = {'7AM': 0,'10AM': 1,'2PM': 2,'6PM': 3,'10PM': 4}
    expiration_map = {'1d': 0,'2h': 1}
    age_map = {'below21': 0,'21': 1,'26': 2,'31': 3,'36': 4,'41': 5,'46': 6,'50plus': 7}
    education_map = {'Some High School': 0,'High School Graduate': 1,
                 'Some college - no degree': 2,'Associates degree': 3,
                 'Bachelors degree': 4,'Graduate degree (Masters or Doctorate)': 5}
    income_map = {'Less than $12500': 0,
              '$12500 - $24999': 1,
              '$25000 - $37499': 2,
              '$37500 - $49999': 3,
              '$50000 - $62499': 4,
              '$62500 - $74999': 5,
              '$75000 - $87499': 6,
              '$87500 - $99999': 7,
              '$100000 or More': 8}
    frequency_map = {'never': 0,
              'less1': 1,
              '1~3': 2,
              '4~8': 3,
              'gt8': 4}
    to_coupon_map = {'5-15': 0,
              '15-25': 1,
              '>25': 2}
    
    for col in frequency_cols:
        df_copy[col] = df_copy[col].map(frequency_map)
    
# map_cols = ['temperature', 'time', 'expiration', 'age', 'education', 'income', 'to_Coupon']
# maps = ['temperature_map', 'time_map', 'expiration_map', 'age_map', 'education_map', 
#         'income_map', 'to_coupon_map']


    df_copy['temperature'] = df_copy['temperature'].map(temperature_map)
    df_copy['time'] = df_copy['time'].map(time_map)
    df_copy['expiration'] = df_copy['expiration'].map(expiration_map)
    df_copy['age'] = df_copy['age'].map(age_map)
    df_copy['education'] = df_copy['education'].map(education_map)
    df_copy['income'] = df_copy['income'].map(income_map)
    df_copy['to_Coupon'] = df_copy['to_Coupon'].map(to_coupon_map)
    
    return df_copy
    


### Encode ordinal categorical features 

In [6]:
X = ordinal_encoding(X)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   destination           12079 non-null  object
 1   passanger             12079 non-null  object
 2   weather               12079 non-null  object
 3   temperature           12079 non-null  int64 
 4   time                  12079 non-null  int64 
 5   coupon                12079 non-null  object
 6   expiration            12079 non-null  int64 
 7   gender                12079 non-null  object
 8   age                   12079 non-null  int64 
 9   maritalStatus         12079 non-null  object
 10  has_children          12079 non-null  int64 
 11  education             12079 non-null  int64 
 12  occupation            12079 non-null  object
 13  income                12079 non-null  int64 
 14  Bar                   12079 non-null  int64 
 15  CoffeeHouse           12079 non-null

### Encode nominal categorical features 

**NOTE**

For nominal feature encoding, use one-hot enccoding. Since ont-hot may generate too many dummies, which lead to complex features and models, some values may be combined.

In [7]:
# maritalStatus
X.maritalStatus.value_counts()
# Combine 'Divorced' and 'Widowed' into one  called 'other'
X['maritalStatus'] = X['maritalStatus'].replace(['Divorced', 'Widowed'], 'Other')
X.maritalStatus.value_counts()


Married partner      4831
Single               4588
Unmarried partner    2048
Other                 612
Name: maritalStatus, dtype: int64

In [8]:
X = pd.get_dummies(X, 
                   columns=['destination', 'passanger', 'weather', 'coupon', 'gender', 
                            'maritalStatus'])
X = X.drop(columns="occupation", axis=1) # drop 'occupation'
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12079 entries, 0 to 12078
Data columns (total 35 columns):
 #   Column                           Non-Null Count  Dtype
---  ------                           --------------  -----
 0   temperature                      12079 non-null  int64
 1   time                             12079 non-null  int64
 2   expiration                       12079 non-null  int64
 3   age                              12079 non-null  int64
 4   has_children                     12079 non-null  int64
 5   education                        12079 non-null  int64
 6   income                           12079 non-null  int64
 7   Bar                              12079 non-null  int64
 8   CoffeeHouse                      12079 non-null  int64
 9   CarryAway                        12079 non-null  int64
 10  RestaurantLessThan20             12079 non-null  int64
 11  Restaurant20To50                 12079 non-null  int64
 12  direction_same                   12079 non-nul

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# y = pd.DataFrame(le.fit_transform(y))

In [9]:
y.value_counts()

1    6877
0    5202
Name: Y, dtype: int64

**NOTE**

The target is rouphly balanced, so accuracy could be used as performance metric. 

## Modeling

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=RANDOM_SEED)

In [62]:
y_test.value_counts()

1    1376
0    1040
Name: Y, dtype: int64

In [11]:
# Classifiers
lr_clf = LogisticRegression(solver='saga', max_iter=1000, random_state=RANDOM_SEED)
dt_clf = DecisionTreeClassifier(random_state=RANDOM_SEED)
rf_clf = RandomForestClassifier(random_state=RANDOM_SEED)
nb_clf = BernoulliNB()
knn_clf = KNeighborsClassifier()
lsvm_clf = LinearSVC(max_iter=5000, dual=False)

# Parameters 
lr_params = dict(C=loguniform(1e-1, 1e2),penalty=['l1', 'l2'])
dt_params = dict(criterion=['gini', 'entropy'],
                 min_samples_split=[2, 4, 6, 8, 10],
                 max_depth=[2, 4, 6, 8, 10])
rf_params = dict(n_estimators=[100, 200, 300],
                criterion=['gini', 'entropy'],
                max_depth=[2, 4, 6, 8, 10],
                min_samples_split=[2, 4, 6, 8, 10])
nb_params = dict(alpha=loguniform(1e-1, 1e0))
knn_params = dict(n_neighbors=[2, 4, 6, 8, 10, 12, 14, 20],
               weights=['uniform', 'distance'])
lsvm_params = dict(C=loguniform(1e-1, 1e2))


clf_list = [lr_clf, dt_clf, rf_clf, nb_clf, knn_clf, lsvm_clf]
params_list = [lr_params, dt_params, rf_params, nb_params, knn_params, lsvm_params]
model_suffix = ['_lr', '_dt', '_rf', '_nb', '_knn', '_lsvm']
model_names = ['Logistic Regression', 'Decison Tree', 'Random Forest', 'Bernoulli Naive Bayes',
               'KNN Classifier', 'Linear SVM']

In [12]:
def train_model_rscv(X_train, y_train, model_suffix=model_suffix, clf_list=clf_list, params_list=params_list):
    for i in range(len(clf_list)):
        # model training with RandomizedSearchCV
        rscv = RandomizedSearchCV(estimator=clf_list[i],
                                  param_distributions=params_list[i],
                                  cv=10, n_jobs=-1, random_state=RANDOM_SEED)
        rscv.fit(X_train, y_train)
        # store cv results
        globals()['rscv%s' % model_suffix[i]] = pd.DataFrame(rscv.cv_results_)
        # store the best model
        globals()['best%s' % model_suffix[i]] = rscv

In [13]:
def get_df_best(cv_result, model_suffix):
    # store the best results into a dataframe
    for i in range(len(cv_result)):
        globals()['df%s' % model_suffix[i]] = cv_result[i].query('rank_test_score == 1')[['params', 'mean_test_score', 'std_test_score']]

In [63]:
def eval_model(X_test, y_test, clf_list, model_names):
    acc = []
    f1_score = []
    #auc_score = []
    for clf in clf_list:
        acc.append(clf.score(X_test, y_test))
        f1_score.append(metrics.f1_score(y_test, clf.predict(X_test)))
#         fpr, tpr, thresholds = metrics.roc_curve(y_test, clf.predict(X_test), pos_label=2)
#         auc_score.append(metrics.auc(fpr, tpr))
    return pd.DataFrame(data={'Model': model_names, 'Accuracy': acc, 'F1_score': f1_score}) #, 'AUC': auc_score

In [15]:
# train models
train_model_rscv(X_train, y_train)

cv_results = [rscv_lr, rscv_dt, rscv_rf, rscv_nb, rscv_knn, rscv_lsvm]
get_df_best(cv_results, model_suffix)

# output the best models
df_list = [df_lr, df_dt, df_rf, df_nb, df_knn, df_lsvm]
for df, model in zip(df_list, model_names):
    df['model'] = model
df_best_model = pd.concat(df_list)

# output evaluation results
best_clfs = [best_lr, best_dt, best_rf, best_nb, best_knn, best_lsvm]
df_eval = eval_model(X_test, y_test, best_clfs, model_names)

/Users/anqizhu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anqizhu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anqizhu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anqizhu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/anqizhu/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_sag.py:352: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [16]:
df_best_model

,params,mean_test_score,std_test_score,model
2,"{'C': 6.251373574521747, 'penalty': 'l1'}",0.678878,0.011710,Logistic Regression
5,"{'C': 1.0025956902289566, 'penalty': 'l2'}",0.678878,0.011710,Logistic Regression
4,"{'min_samples_split': 6, 'max_depth': 8, 'crit...",0.697919,0.010244,Decison Tree
6,"{'n_estimators': 200, 'min_samples_split': 4, ...",0.729690,0.012493,Random Forest
0,{'alpha': 0.2368863950364078},0.652487,0.009260,Bernoulli Naive Bayes
3,{'alpha': 0.3968793330444371},0.652487,0.009260,Bernoulli Naive Bayes
4,{'alpha': 0.1432249371823025},0.652487,0.009260,Bernoulli Naive Bayes
5,{'alpha': 0.14321698289111515},0.652487,0.009260,Bernoulli Naive Bayes
6,{'alpha': 0.1143098387631322},0.652487,0.009260,Bernoulli Naive Bayes
8,{'alpha': 0.39913058785616784},0.652487,0.009260,Bernoulli Naive Bayes


In [17]:
df_eval

,Model,Accuracy,F1_score
0,Logistic Regression,0.689983,0.739840
1,Decison Tree,0.681705,0.732893
2,Random Forest,0.739652,0.788287
3,Bernoulli Naive Bayes,0.655215,0.701327
4,KNN Classifier,0.682533,0.739912
5,Linear SVM,0.689983,0.740381


**NOTE**

Random Forest perfroms the best.

## Boosting

In [27]:
df_best_model.iloc[3, 0] # parameters of random forest

{'n_estimators': 200,
 'min_samples_split': 4,
 'max_depth': 10,
 'criterion': 'gini'}

In [37]:
# Classifiers
xg_clf = XGBClassifier(random_state=RANDOM_SEED)
cat_clf = CatBoostClassifier(random_state=RANDOM_SEED)

# Parameters 
xg_params = dict(max_depth=[8, 10, 12],
                  n_estimators=[100, 200, 300])
cat_params = dict(depth=[8, 10, 12],
                  n_estimators=[100, 200, 300])

clf_list = [xg_clf, cat_clf]
params_list = [xg_params, cat_params]
model_suffix = ['_xg', '_cat']
model_names = ['XG Boosting', 'Cat Boosting']

In [39]:
def train_model_gscv(X_train, y_train, model_suffix=model_suffix, clf_list=clf_list, params_list=params_list):
    for i in range(len(clf_list)):
        # train models with GridSearchCV
        gscv = GridSearchCV(clf_list[i], 
                            param_grid=params_list[i], 
                            cv=10, n_jobs=-1)
        gscv.fit(X_train, y_train)

        # store cv results
        globals()['gscv%s' % model_suffix[i]] = pd.DataFrame(gscv.cv_results_)
        # store the best model
        globals()['best%s' % model_suffix[i]] = gscv

In [35]:
X_train = X_train.rename(columns={'coupon_Restaurant(<20)': 'coupon_Restaurant(sma20)'})
X_test = X_test.rename(columns={'coupon_Restaurant(<20)': 'coupon_Restaurant(sma20)'})

In [40]:
# train models
train_model_gscv(X_train, y_train)

gscv_results = [gscv_xg, gscv_cat]
get_df_best(gscv_results, model_suffix)

# output the best models
df_list_boo = [df_xg, df_cat]
for df, model in zip(df_list_boo, model_names):
    df['model'] = model
df_best_boo = pd.concat(df_list_boo)

# output evaluation results
best_boos = [best_xg, best_cat]
df_eval_boo = eval_model(X_test, y_test, best_boos, model_names)

Learning rate set to 0.214301
0:	learn: 0.6519797	total: 72.4ms	remaining: 7.17s
1:	learn: 0.6222757	total: 81.3ms	remaining: 3.98s
2:	learn: 0.5971439	total: 92.9ms	remaining: 3s
3:	learn: 0.5793810	total: 104ms	remaining: 2.5s
4:	learn: 0.5696442	total: 112ms	remaining: 2.12s
5:	learn: 0.5611128	total: 120ms	remaining: 1.88s
6:	learn: 0.5515535	total: 130ms	remaining: 1.73s
7:	learn: 0.5385881	total: 139ms	remaining: 1.59s
8:	learn: 0.5295156	total: 147ms	remaining: 1.48s
9:	learn: 0.5190559	total: 155ms	remaining: 1.39s
10:	learn: 0.5123105	total: 166ms	remaining: 1.34s
11:	learn: 0.5032913	total: 175ms	remaining: 1.28s
12:	learn: 0.4968899	total: 184ms	remaining: 1.23s
13:	learn: 0.4911861	total: 193ms	remaining: 1.19s
14:	learn: 0.4875056	total: 203ms	remaining: 1.15s
15:	learn: 0.4830897	total: 215ms	remaining: 1.13s
16:	learn: 0.4753985	total: 226ms	remaining: 1.1s
17:	learn: 0.4727486	total: 236ms	remaining: 1.08s
18:	learn: 0.4684252	total: 247ms	remaining: 1.05s
19:	learn: 0.

Learning rate set to 0.214301
0:	learn: 0.6534645	total: 72.4ms	remaining: 7.16s
1:	learn: 0.6223082	total: 82.7ms	remaining: 4.05s
2:	learn: 0.5969678	total: 96.8ms	remaining: 3.13s
3:	learn: 0.5834283	total: 106ms	remaining: 2.54s
4:	learn: 0.5718661	total: 114ms	remaining: 2.16s
5:	learn: 0.5639004	total: 122ms	remaining: 1.91s
6:	learn: 0.5549589	total: 132ms	remaining: 1.76s
7:	learn: 0.5467355	total: 142ms	remaining: 1.63s
8:	learn: 0.5351031	total: 150ms	remaining: 1.51s
9:	learn: 0.5224906	total: 160ms	remaining: 1.44s
10:	learn: 0.5158901	total: 169ms	remaining: 1.37s
11:	learn: 0.5120574	total: 179ms	remaining: 1.31s
12:	learn: 0.5040569	total: 189ms	remaining: 1.26s
13:	learn: 0.4992299	total: 198ms	remaining: 1.22s
14:	learn: 0.4956253	total: 207ms	remaining: 1.17s
15:	learn: 0.4910322	total: 218ms	remaining: 1.15s
16:	learn: 0.4852908	total: 230ms	remaining: 1.12s
17:	learn: 0.4815404	total: 238ms	remaining: 1.09s
18:	learn: 0.4784543	total: 249ms	remaining: 1.06s
19:	lear

17:	learn: 0.5274724	total: 201ms	remaining: 2.04s
18:	learn: 0.5229795	total: 213ms	remaining: 2.03s
19:	learn: 0.5194834	total: 221ms	remaining: 1.99s
20:	learn: 0.5152679	total: 231ms	remaining: 1.97s
21:	learn: 0.5110188	total: 243ms	remaining: 1.96s
22:	learn: 0.5071756	total: 254ms	remaining: 1.96s
23:	learn: 0.5042886	total: 267ms	remaining: 1.96s
24:	learn: 0.5002911	total: 295ms	remaining: 2.06s
25:	learn: 0.4967362	total: 307ms	remaining: 2.05s
26:	learn: 0.4942354	total: 316ms	remaining: 2.03s
27:	learn: 0.4919549	total: 326ms	remaining: 2s
28:	learn: 0.4891033	total: 335ms	remaining: 1.98s
29:	learn: 0.4859181	total: 347ms	remaining: 1.96s
30:	learn: 0.4826201	total: 357ms	remaining: 1.95s
31:	learn: 0.4783016	total: 368ms	remaining: 1.93s
32:	learn: 0.4763570	total: 378ms	remaining: 1.91s
33:	learn: 0.4739056	total: 390ms	remaining: 1.9s
34:	learn: 0.4717650	total: 398ms	remaining: 1.88s
35:	learn: 0.4688487	total: 408ms	remaining: 1.86s
36:	learn: 0.4663916	total: 420ms	r

205:	learn: 0.2920601	total: 2.33s	remaining: 1.06s
206:	learn: 0.2911668	total: 2.35s	remaining: 1.05s
207:	learn: 0.2902291	total: 2.36s	remaining: 1.04s
208:	learn: 0.2893603	total: 2.37s	remaining: 1.03s
209:	learn: 0.2885344	total: 2.38s	remaining: 1.02s
210:	learn: 0.2879551	total: 2.39s	remaining: 1.01s
211:	learn: 0.2869608	total: 2.4s	remaining: 997ms
212:	learn: 0.2859808	total: 2.42s	remaining: 987ms
213:	learn: 0.2847838	total: 2.42s	remaining: 975ms
214:	learn: 0.2843371	total: 2.44s	remaining: 963ms
215:	learn: 0.2834167	total: 2.45s	remaining: 952ms
216:	learn: 0.2827630	total: 2.46s	remaining: 943ms
217:	learn: 0.2822961	total: 2.48s	remaining: 931ms
218:	learn: 0.2815421	total: 2.49s	remaining: 920ms
219:	learn: 0.2809433	total: 2.5s	remaining: 910ms
220:	learn: 0.2799836	total: 2.51s	remaining: 899ms
221:	learn: 0.2792012	total: 2.53s	remaining: 889ms
222:	learn: 0.2784164	total: 2.54s	remaining: 876ms
223:	learn: 0.2779241	total: 2.55s	remaining: 865ms
224:	learn: 0.

153:	learn: 0.2802897	total: 1.81s	remaining: 540ms
154:	learn: 0.2792954	total: 1.82s	remaining: 528ms
155:	learn: 0.2779744	total: 1.83s	remaining: 515ms
156:	learn: 0.2773874	total: 1.84s	remaining: 504ms
157:	learn: 0.2757638	total: 1.85s	remaining: 493ms
158:	learn: 0.2749555	total: 1.86s	remaining: 480ms
159:	learn: 0.2739355	total: 1.87s	remaining: 469ms
160:	learn: 0.2724735	total: 1.88s	remaining: 456ms
161:	learn: 0.2716319	total: 1.89s	remaining: 444ms
162:	learn: 0.2705310	total: 1.9s	remaining: 432ms
163:	learn: 0.2694465	total: 1.91s	remaining: 420ms
164:	learn: 0.2685361	total: 1.92s	remaining: 408ms
165:	learn: 0.2678992	total: 1.93s	remaining: 396ms
166:	learn: 0.2667193	total: 1.95s	remaining: 385ms
167:	learn: 0.2655457	total: 1.96s	remaining: 373ms
168:	learn: 0.2642951	total: 1.97s	remaining: 361ms
169:	learn: 0.2636262	total: 1.98s	remaining: 349ms
170:	learn: 0.2627224	total: 1.99s	remaining: 337ms
171:	learn: 0.2617566	total: 2s	remaining: 325ms
172:	learn: 0.26

...

90:	learn: 0.0503271	total: 8.71s	remaining: 861ms
91:	learn: 0.0496675	total: 8.79s	remaining: 764ms
92:	learn: 0.0488348	total: 8.87s	remaining: 667ms
93:	learn: 0.0480901	total: 8.96s	remaining: 572ms
94:	learn: 0.0475212	total: 9.04s	remaining: 476ms
95:	learn: 0.0469876	total: 9.13s	remaining: 380ms
96:	learn: 0.0463849	total: 9.27s	remaining: 287ms
97:	learn: 0.0454905	total: 9.37s	remaining: 191ms
98:	learn: 0.0450183	total: 9.5s	remaining: 96ms
99:	learn: 0.0445054	total: 9.62s	remaining: 0us
Learning rate set to 0.113501
0:	learn: 0.6565305	total: 99.6ms	remaining: 19.8s
1:	learn: 0.6236454	total: 192ms	remaining: 19s
2:	learn: 0.5965125	total: 274ms	remaining: 18s
3:	learn: 0.5719143	total: 362ms	remaining: 17.7s
4:	learn: 0.5474370	total: 443ms	remaining: 17.3s
5:	learn: 0.5280338	total: 547ms	remaining: 17.7s
6:	learn: 0.5103099	total: 664ms	remaining: 18.3s
7:	learn: 0.4976739	total: 844ms	remaining: 20.3s
8:	learn: 0.4805096	total: 941ms	remaining: 20s
9:	learn: 0.4670019

106:	learn: 0.1546464	total: 9.5s	remaining: 17.1s
107:	learn: 0.1531456	total: 9.63s	remaining: 17.1s
108:	learn: 0.1510064	total: 9.71s	remaining: 17s
109:	learn: 0.1492741	total: 9.79s	remaining: 16.9s
110:	learn: 0.1479901	total: 9.88s	remaining: 16.8s
111:	learn: 0.1464995	total: 9.96s	remaining: 16.7s
112:	learn: 0.1447271	total: 10.1s	remaining: 16.7s
113:	learn: 0.1436898	total: 10.1s	remaining: 16.6s
114:	learn: 0.1420263	total: 10.2s	remaining: 16.5s
115:	learn: 0.1404788	total: 10.3s	remaining: 16.4s
116:	learn: 0.1391677	total: 10.4s	remaining: 16.3s
117:	learn: 0.1383225	total: 10.5s	remaining: 16.2s
118:	learn: 0.1372553	total: 10.6s	remaining: 16.1s
119:	learn: 0.1363614	total: 10.7s	remaining: 16s
120:	learn: 0.1348375	total: 10.8s	remaining: 15.9s
121:	learn: 0.1338285	total: 10.8s	remaining: 15.8s
122:	learn: 0.1325379	total: 10.9s	remaining: 15.7s
123:	learn: 0.1313505	total: 11s	remaining: 15.6s
124:	learn: 0.1298336	total: 11.1s	remaining: 15.6s
125:	learn: 0.12856

107:	learn: 0.1508392	total: 9.63s	remaining: 17.1s
108:	learn: 0.1494176	total: 9.72s	remaining: 17s
109:	learn: 0.1479487	total: 9.8s	remaining: 16.9s
110:	learn: 0.1470571	total: 9.87s	remaining: 16.8s
111:	learn: 0.1452997	total: 9.95s	remaining: 16.7s
112:	learn: 0.1436601	total: 10s	remaining: 16.6s
113:	learn: 0.1424724	total: 10.1s	remaining: 16.5s
114:	learn: 0.1412068	total: 10.2s	remaining: 16.4s
115:	learn: 0.1399529	total: 10.3s	remaining: 16.4s
116:	learn: 0.1390783	total: 10.4s	remaining: 16.3s
117:	learn: 0.1378273	total: 10.5s	remaining: 16.2s
118:	learn: 0.1363117	total: 10.6s	remaining: 16.1s
119:	learn: 0.1352527	total: 10.7s	remaining: 16s
120:	learn: 0.1339435	total: 10.8s	remaining: 16s
121:	learn: 0.1328128	total: 10.9s	remaining: 15.9s
122:	learn: 0.1314812	total: 10.9s	remaining: 15.8s
123:	learn: 0.1297909	total: 11s	remaining: 15.7s
124:	learn: 0.1288215	total: 11.1s	remaining: 15.6s
125:	learn: 0.1279426	total: 11.3s	remaining: 15.5s
126:	learn: 0.1263626	t

Learning rate set to 0.081857
0:	learn: 0.6761940	total: 64.8ms	remaining: 19.4s
1:	learn: 0.6598605	total: 71.3ms	remaining: 10.6s
2:	learn: 0.6459942	total: 77.1ms	remaining: 7.63s
3:	learn: 0.6351869	total: 82.8ms	remaining: 6.13s
4:	learn: 0.6245415	total: 88.9ms	remaining: 5.25s
5:	learn: 0.6180983	total: 95.1ms	remaining: 4.66s
6:	learn: 0.6100718	total: 101ms	remaining: 4.23s
7:	learn: 0.6019001	total: 107ms	remaining: 3.89s
8:	learn: 0.5961580	total: 112ms	remaining: 3.63s
9:	learn: 0.5896787	total: 118ms	remaining: 3.42s
10:	learn: 0.5842915	total: 123ms	remaining: 3.24s
11:	learn: 0.5799528	total: 129ms	remaining: 3.1s
12:	learn: 0.5749557	total: 135ms	remaining: 2.97s
13:	learn: 0.5703524	total: 140ms	remaining: 2.87s
14:	learn: 0.5662731	total: 146ms	remaining: 2.77s
15:	learn: 0.5629478	total: 152ms	remaining: 2.69s
16:	learn: 0.5579809	total: 157ms	remaining: 2.62s
17:	learn: 0.5543806	total: 163ms	remaining: 2.55s
18:	learn: 0.5509780	total: 169ms	remaining: 2.5s
19:	lea

193:	learn: 0.3070690	total: 1.2s	remaining: 655ms
194:	learn: 0.3061689	total: 1.2s	remaining: 648ms
195:	learn: 0.3054604	total: 1.21s	remaining: 642ms
196:	learn: 0.3046511	total: 1.22s	remaining: 636ms
197:	learn: 0.3036980	total: 1.22s	remaining: 629ms
198:	learn: 0.3027178	total: 1.23s	remaining: 623ms
199:	learn: 0.3019859	total: 1.23s	remaining: 617ms
200:	learn: 0.3011712	total: 1.24s	remaining: 611ms
201:	learn: 0.3003495	total: 1.25s	remaining: 604ms
202:	learn: 0.2991757	total: 1.25s	remaining: 598ms
203:	learn: 0.2985856	total: 1.26s	remaining: 592ms
204:	learn: 0.2977030	total: 1.26s	remaining: 585ms
205:	learn: 0.2970114	total: 1.27s	remaining: 579ms
206:	learn: 0.2961585	total: 1.27s	remaining: 573ms
207:	learn: 0.2955628	total: 1.28s	remaining: 567ms
208:	learn: 0.2947701	total: 1.29s	remaining: 560ms
209:	learn: 0.2940697	total: 1.29s	remaining: 554ms
210:	learn: 0.2931878	total: 1.3s	remaining: 548ms
211:	learn: 0.2926314	total: 1.3s	remaining: 541ms
212:	learn: 0.29

In [64]:
# output evaluation results
best_boos = [best_xg, best_cat]
df_eval_boo = eval_model(X_test, y_test, best_boos, model_names)

In [65]:
df_best_boo

,params,mean_test_score,std_test_score,model
5,"{'max_depth': 10, 'n_estimators': 300}",0.761670,0.015685,XG Boosting
2,"{'depth': 8, 'n_estimators': 300}",0.769429,0.016994,Cat Boosting


In [66]:
df_eval_boo

,Model,Accuracy,F1_score
0,XG Boosting,0.763245,0.798165
1,Cat Boosting,0.772351,0.810083


**NOTE**

As the results show, Cat Boosting performs the best. And the acuuracy is highly greater than basic models like logistics regression. 